In [5]:
import numpy as np
import cv2

def detect_colors():
    # Capturing video through webcam
    webcam = cv2.VideoCapture(0)
    # Start a while loop
    while True:
        # Reading the video from the webcam in image frames
        _, imageFrame = webcam.read()

        # Convert the imageFrame in BGR to HSV color space
        hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)

        # Total number of pixels in the frame
        frame_area = imageFrame.shape[0] * imageFrame.shape[1]

        # Define ranges for various colors
        color_ranges = {
            "Red": ([136, 87, 111], [180, 255, 255], (0, 0, 255)),
            "Green": ([25, 52, 72], [102, 255, 255], (0, 255, 0)),
            "Blue": ([94, 80, 2], [120, 255, 255], (255, 0, 0)),
            "Yellow": ([20, 100, 100], [30, 255, 255], (0, 255, 255)),
            "Orange": ([10, 100, 100], [20, 255, 255], (0, 165, 255)),
            "Purple": ([125, 50, 75], [150, 255, 255], (128, 0, 128)),
            "Pink": ([160, 100, 100], [170, 255, 255], (255, 20, 147)),
            "Cyan": ([80, 100, 100], [94, 255, 255], (0, 255, 255)),
            "Brown": ([10, 100, 20], [20, 255, 200], (42, 42, 165)),
            "Gray": ([0, 0, 50], [180, 50, 200], (169, 169, 169)),
            "White": ([0, 0, 200], [180, 55, 255], (255, 255, 255)),
            "Black": ([0, 0, 0], [180, 255, 50], (0, 0, 0)),
        }

        # Kernal for morphological transformation
        kernal = np.ones((5, 5), "uint8")

        # Dictionary to store color percentages
        color_percentages = {}

        # Detect and process each color
        for color_name, (lower, upper, box_color) in color_ranges.items():
            # Create mask for the color
            lower = np.array(lower, np.uint8)
            upper = np.array(upper, np.uint8)
            mask = cv2.inRange(hsvFrame, lower, upper)
            mask = cv2.dilate(mask, kernal)

            # Calculate color percentage
            color_pixels = cv2.countNonZero(mask)
            percentage = (color_pixels / frame_area) * 100
            color_percentages[color_name] = percentage

            # Find contours and draw bounding boxes
            contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 300:
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(imageFrame, (x, y), (x + w, y + h), box_color, 2)
                    cv2.putText(imageFrame, f"{color_name}", (x, y),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.0, box_color)

        # Display the color percentages on the frame
        y_offset = 30
        for color_name, percentage in color_percentages.items():
            cv2.putText(imageFrame, f"{color_name}: {percentage:.2f}%", (10, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            y_offset += 20

        # Display the resulting frame
        cv2.imshow("Color Detection with Percentages", imageFrame)

        # Check for key presses
        key = cv2.waitKey(10)
        if key & 0xFF == ord('q'):  # Quit on pressing 'q'
            print("Camera stopped.")
            webcam.release()
            cv2.destroyAllWindows()
            break
        elif key & 0xFF == ord('w'):  # Turn off camera on pressing 'w'
            print("Camera turned off. Press 'q' to exit.")
            webcam.release()
            cv2.destroyAllWindows()
            break

# Call the function
detect_colors()

Camera turned off. Press 'q' to exit.
